# Jupyter Note book MoMo Open API SandBox

In [30]:
from datetime import datetime
global Api_User 
global Api_Key 
GetPaid_Debit_Request_Ref_ID = "" #UUID String Request Reference 
Pay_Transfer_Request_Ref_ID = "" #UUID String Request Reference
Token=""
Token_expiry_time = ""
Token_expired = False
Token_expiry_time = datetime.now()
Environment = "sandbox" #Target Environment  
Collection_Subscription_Primary_Key  = "4c91dae7a6f1474387a23a1f3d448eb7"#Primary Key for Collection Subscription.https://momodeveloper.mtn.com/profile
Disbursement_Subscription_Primary_Key  = "dec90f29f4e14137912bfc3236d51cbe"#Primary Key for Disbursement Subscription.https://momodeveloper.mtn.com/profile
Base_Url = "https://sandbox.momodeveloper.mtn.com" #SandBox Base URL

# timer-wait-time

In [31]:
import time

def countdown(seconds):
    while seconds >= 1:
        print(str(seconds), end= ':')
        time.sleep(1)
        seconds -= 1
    print("0")



# Creating API User on the SandBox

In [3]:
#Function to create an API User (Username) from The MoMo OpenAPI SandBox
def Create_API_User_SandBox():
    import requests as rq
    import uuid
    global Api_User
    global Collection_Subscription_Primary_Key
    Api_User = str(uuid.uuid4())
    url = Base_Url+"/v1_0/apiuser"
    headers = {
    "X-Reference-Id": Api_User, #When creating Api user, the X Reference Id in the header will be created as the user. 
    "Ocp-Apim-Subscription-Key": Collection_Subscription_Primary_Key,
    "Content-Type": "application/json"
    }
    body = {    
    "providerCallbackHost": "webhook.site" # If your callback is https://webhook.site/mycallback/site then the providerCallbackHost is webhook.site
    }
    try:
        resp = rq.request("post", url, json=body, headers=headers)
        if(str(resp.status_code)=="201"):
            print("HTTP Status Code:"+str(resp.status_code)+"\n Api user Created: "+Api_User)
        elif(str(resp.status_code)=="401"):
            print(str(resp.status_code)+" "+resp.text+" ")
            print("Ensure the subscription key is the primary")
        elif(str(resp.status_code)=="400"):
            print(str(resp.status_code)+" "+resp.text+" ")
            print("Ensure API User(X-Reference-Id) in the Headers is UUID Version 4")
            print("Ensure the Body contains the correct syntax ""\"providerCallbackHost""\""+":"+"Your CallBack URL HOST Eg ""\"webhook.site""\"")
        else:
            print(str(resp.status_code)+" "+resp.text+" ")
    except TypeError:
        print("Body of the Request has to be Json Format")
    except:
        print("Something Is Wrong "+resp.json)
#Create_API_User_SandBox()        

#### <b>Creating the API Key (Password) For the API USER Created SandBox  {{Api_User}}</b>

In [ ]:
#Function to create an API Key for the API User from The MoMo OpenAPI SandBox
def Create_API_Key_SandBox():
    import requests as rq
    import traceback
    global Api_Key
    global Collection_Subscription_Primary_Key
    url = Base_Url+"/v1_0/apiuser/"+Api_User+"/apikey"
    headers = {
    "Ocp-Apim-Subscription-Key": Collection_Subscription_Primary_Key,
    }    
    try:
        resp = rq.request("post", url, headers=headers)
        if(str(resp.status_code)=="201"):
            Response = resp.json()
            Api_Key = Response.get('apiKey')#Save the API Key in Variable 
            print("HTTP Status Code:"+str(resp.status_code)+"\n Api User:"+str(Api_User) +" Api Key:"+str(Api_Key))
        elif(str(resp.status_code)=="400"):
            print(str(resp.status_code)+" "+resp.text+" Validate the BaseURL \n And Ensure API_User is created, by calling the function Create_API_User_SandBox()")
        elif(str(resp.status_code)=="404"):
            print(str(resp.status_code)+" "+resp.text+" API_USER was not created, Please Run function Create_API_User_SandBox()")
        else:
            print(str(resp.status_code)+" "+resp.text+" ")
    except TypeError:
        print("Body of the Request has to be Json Format or No Body")
    except:
        print("Something Is Wrong ")
        traceback.print_exc()      
#Create_API_Key_SandBox()

##### <b>NOTE: API User and API Key are created once, but advisable to have the API Key changed periodically</b>

In [32]:
Create_API_User_SandBox() #Function create to create API User and store value in Variable {{Api_User}}
Create_API_Key_SandBox() #Function to create API Key(Password) for the Api User Created and stored in variable {{Api_Key}}


HTTP Status Code:201
 Api user Created: 1d1aafa6-39f2-4f89-904d-930d9937f779
HTTP Status Code:201
 Api User:1d1aafa6-39f2-4f89-904d-930d9937f779 Api Key:3dc8ea0cd42343f5924dc280965adabf


#### <b>With the API User and API Key now set up, the next step is to generate a Bearer Token. <br>
This token, which is necessary for authenticating most upcoming API requests, <br>
will expire one hour after its creation.</b>

In [6]:
#Function to generate Token and set Token expiry. 
def Get_Token():# function to return token (renews token if expired)
    import requests as rq
    import traceback
    import json
    from datetime import datetime, timedelta
    global Token 
    global Token_expiry_time
    EndPoint = Base_Url+"/collection/token/"
    Auth = bytes(Api_User + ':' + Api_Key, "utf-8")
    headers = {    
    "Ocp-Apim-Subscription-Key": Collection_Subscription_Primary_Key,
    }
    try:
        resp = rq.request("post", EndPoint,auth=(Api_User,Api_Key), headers=headers)
        Response = resp.json()
        if(str(resp.status_code) == "200"):
            Token = Response.get('access_token')
            Token_expiry = Response.get('expires_in')
            Token_expiry_time = datetime.now() + timedelta(seconds= int(Token_expiry)) #Track Token Expiry Time 
            print("New Token Generated Expiring at :" +str(Token_expiry_time))           
        elif(str(resp.status_code) == "500" or str(Response.get("error"))=="login_failed"):
            print(Response)
            print("Ensure to Map the API User and API Key as (Username:Password) respectively")
        else:
            print(resp.text)            
    except:
        print("Something Is Wrong ")
        traceback.print_exc()  
#Get_Token()

#### <b>To prevent the creation of tokens for each request,<br>
 it is advisable to verify the token's validity and only generate a new one when the previous token has expired.</b>

In [ ]:
#Function to Validate  Status of Token
#If the Token is Expired a new one will be generated. 
def Token_Status():
    if Token_expiry_time >= datetime.now():
        Token_expired = False    
        #print ("Token not Expired: Expiring at "+ str(Token_expiry_time))
        #print(Token)
    else:
        Token_expired = True
        Get_Token()
        #print ("New Token Generated Expiring at "+ str(Token_expiry_time))
        #print(Token)
##Token_Status()

#### <b>Get Paid (Debit API ) Function

In [8]:
#Function that initiates a Debit USSD Prompt to the Payer to approve wit PIN
def Request_Debit_Payment(MSISDN,Amount):
  import requests as rq
  from datetime import datetime, timedelta
  import traceback
  import uuid
  global GetPaid_Debit_Request_Ref_ID
  Token_Status()
  GetPaid_Debit_Request_Ref_ID = str(uuid.uuid4())
  url = Base_Url+"/collection/v1_0/requesttopay"
  headers = {
    "X-Reference-Id": GetPaid_Debit_Request_Ref_ID, #Unique for every request, used to validate status of the request. 
    "X-Target-Environment": Environment,
    "Ocp-Apim-Subscription-Key": Collection_Subscription_Primary_Key,
    "Authorization":"Bearer "+Token, #Avoid creating new tokens for every request,  track the Expiry 
    "Content-Type": "application/json",
    "X-Callback-Url":"https://webhook.site/mycallback/site"### You can add X-Callback-Url to receive the callback  ("X-Callback-Url":"https://webhook.com/mysite/status")
  }
  body = {    
    "amount": Amount,
    "currency": "EUR", #use the currency as EUR in the SandBox
    "externalId": str(uuid.uuid1()), #Used for Reconciliation between application and MoMo platform. 
    "payer": {
      "partyIdType": "MSISDN",#EMAIL and ALIAS apply as well 
      "partyId": MSISDN
  },
    "payerMessage": "MoMo Debit API", #Message sent to the Payer
    "payeeNote": "MoMo Debit API" #Message Note to the  Payee
  }
  try:
    resp = rq.request("post", url, json=body, headers=headers)
    if(str(resp.status_code) == "202"):
      print("Debit request to MSISDN "+MSISDN+" Amount "+Amount+" "+ "Response Code "+str(resp.status_code))
      countdown(6)
      print("Request_Reference_ID :"+GetPaid_Debit_Request_Ref_ID )
    elif (str(resp.status_code) == "404"):
      print("Check The Base_URL ")
    elif (str(resp.status_code) == "400"):
      print("Ensure no Special Charters like & in the Message and Notes \nThe X-Reference-Id in the header should be UUID Versio 4")
      print(resp.text)
    elif (str(resp.status_code) == "500" or str(resp.json().get("message")).endswith("INVALID_CALLBACK_URL_HOST") or str(resp.json().get("message")).endswith("Currency not supported.")):
      print(resp.json())
      print("Ensure the  URL Host is the same with the one created when generating API_USer function ")
      print("Verify and validate  Currency for Sand Box is EUR")
    elif (str(resp.status_code) == "500" ):
      print(resp.text)
      print("API is not available")
    else:
      print(resp.status_code)
      print(resp.text)
  except TypeError:
    print("Request Body should be Json Formatted")
  except:
    print("Something Is Wrong ")
    traceback.print_exc()   
#Request_Debit_Payment("56733123453","50000")

#### <b>From the Test paryer_Id test numbers, we can  validate the status of the Requests </b>

In [9]:
#Check Status 
def Check_Debit_Status(X_Reference_Id_Of_The_Debit_Request):
    import requests as rq
    import json
    import traceback
    Token_Status()
    global Debit_Status
    url = Base_Url+"/collection/v1_0/requesttopay/"+X_Reference_Id_Of_The_Debit_Request
    headers = {
    "X-Target-Environment": Environment,
    "Ocp-Apim-Subscription-Key": Collection_Subscription_Primary_Key,
    "Authorization":"Bearer "+Token,
    }
    try:
        resp = rq.request("get", url,headers=headers)
        Status_Json = resp.json()
        Status_Json_DD = str(Status_Json).replace('\'', '"')
        Debit_Status = Status_Json.get('status')
        print(str(Status_Json))
        
    except:
        print("Something Is Wrong ")
        traceback.print_exc()       
    #print(Status_Json)
    
#Check_Status("")


#### <b>Sending  a Notification to the Payer after a successful Debit Request. 

In [10]:
#Send a Notification After a succesfull Request to Pay
def Send_Notification (X_Reference_Id_Of_The_Debit_Request):
    import requests as rq
    import json
    import traceback
    Token_Status()
    url = Base_Url+"/collection/v1_0/requesttopay/"+X_Reference_Id_Of_The_Debit_Request+"/deliverynotification"
    headers = {
    "X-Target-Environment": Environment,
    "Ocp-Apim-Subscription-Key": Collection_Subscription_Primary_Key,
    "Authorization":"Bearer "+Token,
    }
    body = {"notificationMessage": "Product has been Successfull sent to your Home, Thank You fro Purchasing from MoMo"}
    
    try:
        resp = rq.request("post", url, json=body, headers=headers)
        print(str(resp.status_code)+":Ok "+" Notification Sent")       
        
    except:
        print("Something Is Wrong ")
        traceback.print_exc()       
    #print(Status_Json)


#### <b>Refund a successful Debit to the Payer

In [39]:
#Send a refund of a debit transaction
def Refund_Debit (X_Reference_Id_Of_The_Debit_Request,Amount):
    import requests as rq
    import json
    import uuid
    import traceback
    global GetPaid_Debit_Refund_Ref_ID
    GetPaid_Debit_Refund_Ref_ID = str(uuid.uuid4())
    Token_Status()
    url = Base_Url+"/disbursement/v1_0/refund"
    headers = {
    "X-Reference-Id": GetPaid_Debit_Refund_Ref_ID,
    "X-Target-Environment": Environment,
    "Ocp-Apim-Subscription-Key": Disbursement_Subscription_Primary_Key,
    "Authorization":"Bearer "+Token
    }
    body = {    
    "amount": str(Amount),
    "currency": "EUR", #use the currency as EUR in the SandBox
    "externalId": str(uuid.uuid1()), #Used for Reconciliation between application and MoMo platform. 
    "payerMessage": "MoMo Reund API", #Message sent to the Payer
    "payeeNote": "MoMo Refund API", #Message Note to the  Payee
    "referenceIdToRefund": str(X_Reference_Id_Of_The_Debit_Request)
  }
    
    try:
        resp = rq.request("post", url, json=body, headers=headers)
        print(str(resp.status_code))
        print(str(resp.text)) 
        
    except:
        print("Something Is Wrong ")
        traceback.print_exc()       
    #print(Status_Json)



##### <b>Get Status of a Refund

In [ ]:
#Check Refund Status
def Check_Debit_Status(X_Reference_Id_Of_The_Debit_Request):
    import requests as rq
    import json
    import traceback
    Token_Status()
    global Debit_Status
    url = Base_Url+"/collection/v1_0/requesttopay/"+X_Reference_Id_Of_The_Debit_Request
    headers = {
    "X-Target-Environment": Environment,
    "Ocp-Apim-Subscription-Key": Collection_Subscription_Primary_Key,
    "Authorization":"Bearer "+Token,
    }
    try:
        resp = rq.request("get", url,headers=headers)
        Status_Json = resp.json()
        Status_Json_DD = str(Status_Json).replace('\'', '"')
        Debit_Status = Status_Json.get('status')
        print(str(Status_Json))
        
    except:
        print("Something Is Wrong ")
        traceback.print_exc()       
    #print(Status_Json)
    
#Check_Status("")

#### <b>Testing Status Responses with payer_Id_msisdn</b>
|  payer_id-Msisdn | Response status 
| --- | --- | 
46733123450	|Failed
46733123451	|Rejected
46733123452	|Timeout
56733123453	|Success
46733123454	|Pending

In [40]:

Request_Debit_Payment("56733123452","50000")
status = Check_Debit_Status(GetPaid_Debit_Request_Ref_ID)
if (Debit_Status == "SUCCESSFUL"):
    
    Send_Notification(GetPaid_Debit_Request_Ref_ID)
    Refund_Debit(GetPaid_Debit_Request_Ref_ID,"200")

else:
    print(Debit_Status+""+" Notification No Sent")

Debit request to MSISDN 56733123452 Amount 50000 Response Code 202
6:5:4:3:2:1:0
Request_Reference_ID :24e8a890-51e6-46e5-9c46-7f7492c6ce2b
{'financialTransactionId': '467989344', 'externalId': '3f890f3a-9e62-11ef-ba86-00155d884321', 'amount': '50000', 'currency': 'EUR', 'payer': {'partyIdType': 'MSISDN', 'partyId': '56733123452'}, 'payerMessage': 'MoMo Debit API', 'payeeNote': 'MoMo Debit API', 'status': 'SUCCESSFUL'}
200:Ok  Notification Sent
202



#### <b>Get Basic Info KYC Function

In [13]:
#Get Basic Info KYC (FirstName and LastName)
def Check_BasicInfo_KYC(Number,NAME):
    import requests as rq
    import json
    import traceback
    global First_Name, Last_Name,Proceed
    Token_Status()
    url = Base_Url+"/disbursement/v1_0/accountholder/msisdn/"+Number+"/basicuserinfo"
    headers = {
    "X-Target-Environment": Environment,
    "Ocp-Apim-Subscription-Key": Disbursement_Subscription_Primary_Key,
    "Authorization":"Bearer "+Token,
    }
    try:
        resp = rq.request("get", url,headers=headers)
        Status_Json = resp.json()
        Status_Json_DD = str(Status_Json).replace('\'', '"')
        First_Name = Status_Json.get("given_name")
        Last_Name = Status_Json.get("family_name")
        if((str(First_Name).upper()+" "+str(Last_Name).upper()) ==NAME):
            Proceed = True
        else:
            Proceed = False        
    except:
        print("Something Is Wrong ")
        traceback.print_exc()       
    #print(Status_Json)
#Check_BasicInfo_KYC("56733123453","BOX SAND")

#### <b>Get Detailed KYC Function  (With Consent)

##### <b>Generate Consent (bc-authorize) Function

In [14]:
#Send a Notification After a succesfull Request to Pay
def bc_authorize ():
    import requests as rq
    import json
    import traceback
    global Consent_ID
    Token_Status()
    url = Base_Url+"/disbursement/v1_0/bc-authorize"
    headers = {
    "X-Target-Environment": Environment,
    "Ocp-Apim-Subscription-Key": Disbursement_Subscription_Primary_Key,
    "Authorization":"Bearer "+Token,
    'Content-Type': 'application/x-www-form-urlencoded'
    }
    body = 'scope=all_info&login_hint=ID:5636675677/MSISDN&access_type=offline'    
    try:
        resp = rq.request("post", url, data=body, headers=headers)
        resp_json = resp.json()
        Consent_ID =resp_json.get("auth_req_id")
        print(Consent_ID )
    except:
        print("Something Is Wrong ")
        traceback.print_exc()       
    #print(Status_Json)
bc_authorize ()

2000018c-7d5f-4549-b522-4d14690ff0da


##### <b>Generate Consent Token (oauth2) Function

In [15]:
#Function to generate Oauth Token and set Token expiry. 
def Get_Oauth_Token(consent_auth_req_id):# function to return token (renews token if expired)
    import requests as rq
    import traceback
    import json
    from datetime import datetime, timedelta
    global Token_Oauth
    global Disbursement_Subscription_Primary_Key
    global Token_Oauth_expiry_time
    EndPoint = Base_Url+"/disbursement/oauth2/token/"
    #Auth = bytes(Api_User + ':' + Api_Key, "utf-8")
    headers = {    
    "Ocp-Apim-Subscription-Key": Disbursement_Subscription_Primary_Key,
    'Content-Type': 'application/x-www-form-urlencoded',
    "X-Target-Environment": Environment
    }
    body = "grant_type=urn:openid:params:grant-type:ciba&auth_req_id="+consent_auth_req_id+""
    try:
        resp = rq.request("POST", EndPoint,auth=(Api_User,Api_Key), headers=headers,data=body)
        Response = resp.json()        
        if(str(resp.status_code) == "200"):
            #print(Response)
            Token_Oauth = Response.get('access_token')
            Token_Oauth_expiry = Response.get('expires_in')
            Token_Oauth_expiry_time = datetime.now() + timedelta(seconds= int(Token_Oauth_expiry)) #Track Token Expiry Time 
            print("New Token Generated Expiring at :" +str(Token_Oauth_expiry_time))           
        elif(str(resp.status_code) == "500" or str(Response.get("error"))=="login_failed"):
            print(Response)
            print("Ensure to Map the API User and API Key as (Username:Password) respectively")
        else:
            print(resp.text+" "+str(resp.status_code))
            print(rq)
    except:
        print("Something Is Wrong ")
        traceback.print_exc()  
#Get_Oauth_Token(Consent_ID)

##### <b>Status Token (oauth2) Function

In [16]:
#Function to Validate  Status of Token
#If the Token is Expired a new one will be generated. 
def Token_Oauth_Status():
    if Token_Oauth_expiry_time >= datetime.now():
        Token_expired = False    
        #print ("Token not Expired: Expiring at "+ str(Token_expiry_time))
        #print(Token)
    else:
        Token_expired = True
        Get_Oauth_Token(Consent_ID)
        #print ("New Token Generated Expiring at "+ str(Token_expiry_time))
        #print(Token)
#Token_Oauth_Status()

##### <b>Get Detailed KYC with Consent Token (oauth2) Function

In [17]:
#Get Basic Info KYC (FirstName and LastName)
def Get_DetailedInfo_KYC():
    import requests as rq
    import json
    import traceback
    Token_Oauth_Status()
    url = Base_Url+"/disbursement/oauth2/v1_0/userinfo"
    headers = {
    "X-Target-Environment": Environment,
    "Ocp-Apim-Subscription-Key": Disbursement_Subscription_Primary_Key,
    "Authorization":"Bearer "+Token_Oauth
    }
    try:
        resp = rq.request("get", url,headers=headers)
        Response = resp.json()
        print(resp.text)       
    except:
        print("Something Is Wrong ")
        traceback.print_exc()       
    #print(Status_Json)
#Get_DetailedInfo_KYC()

### <b>Get Status on Disbursements Transfer Function

In [18]:
#Check Status Disbursements Transfer
def Check_Status_Transfer(X_Reference_Id_Of_The_Transfer_Request):
    import requests as rq
    import json
    import traceback
    Token_Status()
    url = Base_Url+"/disbursement/v1_0/transfer/"+X_Reference_Id_Of_The_Transfer_Request
    headers = {
    "X-Target-Environment": Environment,
    "Ocp-Apim-Subscription-Key": Disbursement_Subscription_Primary_Key,
    "Authorization":"Bearer "+Token,
    }
    try:
        resp = rq.request("get", url,headers=headers)
        Status_Json = resp.json()
        Status_Json_DD = str(Status_Json).replace('\'', '"')
        print(Status_Json)        
    except:
        print("Something Is Wrong ")
        traceback.print_exc()       
    #print(Status_Json)
    
#Check_Status_Transfer("Check_Status_Transfer")

#### <b>Pay - Disbursement  Transfer Function

In [19]:
#Disbursements Transfer
#Function that initiates a Transfer from  Business  Wallet to a customer's  
def Request_Tranfer_Payment(MSISDN,MSISDN_NAME,Amount):
  import requests as rq
  from datetime import datetime, timedelta
  import traceback
  import uuid
  global Pay_Transfer_Request_Ref_ID 
  Token_Status()
  Check_BasicInfo_KYC(MSISDN,MSISDN_NAME)
  if(Proceed==True):
    Pay_Transfer_Request_Ref_ID  = str(uuid.uuid4())  
    url = Base_Url+"/disbursement/v1_0/transfer"
    headers = {
      "X-Reference-Id": Pay_Transfer_Request_Ref_ID , #Unique for every request, used to validate status of the request. 
      "X-Target-Environment": Environment,
      "Ocp-Apim-Subscription-Key": Disbursement_Subscription_Primary_Key ,
      "Authorization":"Bearer "+Token, #Avoid creating new tokens for every request,  track the Expiry 
      "Content-Type": "application/json",
      "X-Callback-Url":"https://webhook.site/mycallback/site"### You can add X-Callback-Url to receive the callback  ("X-Callback-Url":"https://webhook.com/mysite/status")
    }
    body = {    
      "amount": Amount,
      "currency": "EUR", #use the currency as EUR in the SandBox
      "externalId": str(uuid.uuid1()), #Used for Reconciliation between application and MoMo platform. 
      "payee": {
      "partyIdType": "MSISDN",#EMAIL and ALIAS apply as well 
      "partyId": MSISDN
    },
      "payerMessage": "MoMo Debit API", #Message sent to the Payer
      "payeeNote": "MoMo Debit API" #Message Note to the  Payee
    }
    try:
      resp = rq.request("post", url, json=body, headers=headers)
      if(str(resp.status_code) == "202"):
        print("Transfer request "+Pay_Transfer_Request_Ref_ID+" to MSISDN "+MSISDN+" Amount "+Amount+" "+ "Response Code "+str(resp.status_code))
        countdown(6)
        Check_Status_Transfer(Pay_Transfer_Request_Ref_ID)
        print("Completed")        
      elif (str(resp.status_code) == "404"):
        print("Check The Base_URL ")
      elif (str(resp.status_code) == "400"):
        print("Ensure no Special Charters like & in the Message and Notes \nThe X-Reference-Id in the header should be UUID Versio 4")
        print(resp.text)
      elif (str(resp.status_code) == "500" or str(resp.json().get("message")).endswith("INVALID_CALLBACK_URL_HOST") or str(resp.json().get("message")).endswith("Currency not supported.")):
        print(resp.json())
        print("Ensure the  URL Host is the same with the one created when generating API_USer function ")
        print("Verify and validate  Currency for Sand Box is EUR")
      elif (str(resp.status_code) == "500" ):
        print(resp.text)
        print("API is not available")
      else:
        print(resp.status_code)
        print(resp.text)
    except TypeError:
      print("Request Body should be Json Formatted")
    except:
      print("Something Is Wrong ")
      traceback.print_exc()   
  else:
    print("Names do not match")
#Request_Tranfer_Payment("56733123453","50000")

##### <b>Testing Status Responses with payee_Id_msisdn</b>
|  payee_id-Msisdn | Response status 
| --- | --- | 
46733123450	|Failed
46733123451	|Rejected
46733123452	|Timeout
56733123453	|Success
46733123454	|Pending
#####  Each Payee ID MSISDN will give a static response 

In [20]:
Request_Tranfer_Payment("56733123453","SAND BOX","500")

Transfer request 360a1680-15e1-47fe-9e00-e7d6f84efdf3 to MSISDN 56733123453 Amount 500 Response Code 202
6:5:4:3:2:1:0
{'amount': '500', 'currency': 'EUR', 'financialTransactionId': '1038770800', 'externalId': '7ca6d5bc-9e3b-11ef-8abc-00155d884321', 'payee': {'partyIdType': 'MSISDN', 'partyId': '56733123453'}, 'payerMessage': 'MoMo Debit API', 'payeeNote': 'MoMo Debit API', 'status': 'SUCCESSFUL'}
Completed
